# Nuberu experimental results

## Reading data

In [ ]:
data_file = "../nuberu-starter-lab/notebooks/all_requests_data.parquet"

In [ ]:
import pandas as pd
data = pd.read_parquet(data_file)
data['injected'] = pd.to_datetime(data['injected'], unit='s', origin='2025-06-24 09:00:00')

## Summary of results



In [ ]:
# Compute the percentage of finished requests for each app
percentage_pivot = data.pivot_table(
    index=['queue_size', 'termination', 'LB', 'distribution'],
    columns='app', 
    values='finished',
    aggfunc=lambda x: x.sum() / len(x) * 100,
    fill_value=0
).round(1)

# Append the cost of each experiment
cost_summary = data.groupby(['queue_size', 'termination', 'LB', 'distribution'])['cost'].first()
summary_with_cost = percentage_pivot.join(cost_summary)

summary_with_cost

## Plots

In [ ]:
# Setup for interactive widgets
from ipywidgets import interact, Dropdown, HBox, VBox, Output, Button, HTML
from IPython.display import display, clear_output, Markdown

distribution_options = ["trace", "poisson"]
termination_options = ["hard", "drain"]
lb_options = ["weighted", "simple"]
queue_size_options = ["q1000", "q0"]
app_options = ["app0", "app1", None]
plot_options = ["per app", "per container"]

distribution_selector_j = Dropdown(options=distribution_options, value="poisson", description="Distribution")
termination_selector_j = Dropdown(options=termination_options, value="hard", description="Termination")
lb_selector_j = Dropdown(options=lb_options, value="weighted", description="LB")
queue_selector_j = Dropdown(options=queue_size_options, value="q1000", description="Q")
app_selector_j = Dropdown(options=app_options, value="app0", description="App")
plot_selector_j = Dropdown(options=plot_options, value="per container", description="Plot kind")
run_button = Button(
    description='Run Analysis',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to run the analysis with the selected options',
    icon='check'
)

controls = VBox([
    HTML(value="<h3>Experiment Parameters</h3>"),
    HBox([distribution_selector_j, termination_selector_j]),
    HBox([lb_selector_j, queue_selector_j]),
    HTML(value="<h3>Analysis Options</h3>"),
    HBox([app_selector_j, plot_selector_j]),
    run_button
])
output_area = Output()
output_area = Output()
# Functions for selecting data and creating plots
def filter_data(data_df, experiment=None):
    if experiment is not None:
        data_df = data_df[data_df["experiment"] == experiment]
    return data_df

def select_experiment_df(data, distribution, termination, lb, queue_size):
    selected_experiment_name = (
        f"{distribution}_"
        f"{termination}_"
        f"{lb}_"
        f"{queue_size}"
    )
    selected_data = filter_data(data, experiment=selected_experiment_name)
    return selected_data

# def update_dashboard(
#     distribution,
#     termination,
#     lb,
#     queue_size,
#     app_selected, 
#     plot_kind_selected
# ):
#     app = app_selected
#     plot_kind = plot_kind_selected

#     if plot_kind == "per app":
#         grouper = "app"
#     else:
#         grouper = "ic-cc"

#     selected_data = select_experiment_df(data, distribution, termination, lb, queue_size)

#     # # --- Aquí iría tu código de ploteo o análisis con 'selected_data_jupyter' ---
#     # # Por ejemplo, para mostrar las primeras filas de los datos filtrados:
#     print(f"Experiment selected: {selected_experiment_name}")
#     print(f"App selected: {app}")
#     print(f"Grouper: {grouper}")
#     print("\nFiltered Data Head:")
#     display(selected_data.head()) # Usamos display para mostrar DataFrames en Jupyter

def on_run_button_clicked(b):
    with output_area:
        clear_output(wait=False)  # Limpia la salida anterior
        _data = select_experiment_df(
            data,
            distribution_selector_j.value,
            termination_selector_j.value,
            lb_selector_j.value,
            queue_selector_j.value
        )
        _data = _data[["injected", "experiment", "app", "status", "service_time", "response_time"]].set_index("injected")
        display(_data.head())
run_button.on_click(on_run_button_clicked)

In [ ]:
display(controls, output_area)